# Interpolation of profiles from data

As noted in the example on constraining with time series, if you want to your simulated turbulence to look like your measured time series when the simulation point is near a measured point, you need to have profile functions that will recreate the values from the data when needed. PyConTurb has a custom interpolator and profiles to handle this. The interpolator is demonstrated in another example; this one will focues on the profile functions that interpolate from the constraining data in `con_tc`.

This example has the following sections:  

* [Preliminaries](#Preliminaries:-importing/defining-functions)  

* [Wind profile interpolated from data](#Wind-profile-interpolated-from-data)  

* [Sig profile interpolated from data](#Sig-profile-interpolated-from-data)  

* [Spectra interpolated from data](#Spectra-interpolated-from-data)  

## Preliminaries: imports and definitions

We first import required functions and set a few notebook-specific functions/variables.

In [ ]:
%matplotlib inline
from pathlib import Path

import matplotlib.pyplot as plt  # matplotlib for some plotting
import numpy as np  # numeric python functions
import pandas as pd  # need this to load our data from the csv files

from pyconturb import TimeConstraint
from pyconturb.sig_models import data_sig
from pyconturb.spectral_models import data_spectrum
from pyconturb.wind_profiles import data_profile

from _nb_utils import plot_interp

# load the constraint file used in the constrained example, used for our "data"
data_dir = Path('.').absolute() / 'data'
con_tc = TimeConstraint(pd.read_csv(data_dir / 'con_tc.csv', index_col=0))  # load data from csv into tc
con_tc.index = con_tc.index.map(lambda x: float(x) if (x not in 'kxyz') else x)  # index cleaning

## Wind profile interpolated from data

We will use the same `TimeConstraint` object as we did in the example on constraining data. The object was loaded in during the "Preliminaries" section above. Recall that it has constraints on $u$, $v$ and $w$ at 6 heights ranging from 115 to 131 m.

Instead of using the `power_profile` or `constant_profile` functions like we did before, we can use the `data_profile` function. Usage of this function is detailed in the Reference Guide, but it's just like the other profile functions except we pass in `con_tc` as a keyword argument.

Here is a "sanity check" comparing the mean wind speed from `con_tc` to the values returned by `data_profile`.

In [ ]:
# define the points we want to query and call data_profile
y, z = np.meshgrid([-50, 50], np.linspace(10, 140))
wsp_intp = data_profile(y.flatten(), z.flatten(), con_tc=con_tc)
# calculate the values fron con_tc for comparison
yp, zp = np.zeros(6), con_tc.loc['z'].unique()
wspp = con_tc.get_time().filter(regex='u_').mean().values
# plot the interpolated values
plot_interp(yp, zp, wspp, y, z, wsp_intp)

## Sig profile interpolated from data

The usage of the `data_sig` function is similar to the `iec_sig` function, except that we must pass in `con_tc` as a keyword argument. Here is a sanity check similar to the one we gave for `wind_profile`.

In [ ]:
# define the points we want to query and call data_sig
y, z = np.meshgrid([-50, 50], np.linspace(10, 140))
plt.figure(1, figsize=(15, 4))
for k in range(3):  # loop through components
    ks = k * np.ones(y.size)
    sig_intp = data_sig(ks, y.flatten(), z.flatten(), con_tc=con_tc)
    # calculate the values fron con_tc for comparison
    yp, zp = np.zeros(6), con_tc.loc['z'].unique()
    sigp = con_tc.get_time().filter(regex=f'{"uvw"[k]}_').std().values
    # plot the interpolated values
    plt.subplot(1, 3, 1 + k)
    plot_interp(yp, zp, sigp, y, z, sig_intp)
    plt.title(f'Component {"uvw"[k]} sig')

## Spectra interpolated from data

Lastly, there is also a function, `data_spectrum`, to interpolate spectral profiles from `con_tc`. Here's an example of how it works.

In [ ]:
# define the points we want to query and call data_spectrum
f, f_idx = 0.05, 5  # frequency to look at and its index in the freq vector
y, z = np.meshgrid([-50, 50], np.linspace(10, 140))
plt.figure(1, figsize=(14, 10))
for k in range(3):  # loop through components
    ks = k * np.ones(y.size)
    spc_intp = data_spectrum(f, ks, y.flatten(), z.flatten(), con_tc=con_tc)
    # calculate the values fron con_tc for comparison
    yp, zp = np.zeros(6), con_tc.loc['z'].unique()
    mags = 2*np.abs(np.fft.rfft(con_tc.get_time().filter(regex=f'{"uvw"[k]}_'), axis=0))**2
    spcp = mags[f_idx, :]
    # plot the interpolated values
    plt.subplot(2, 2, 1 + k)
    plot_interp(yp, zp, spcp, y, z, spc_intp, fmt='%.2e')
    plt.title(f'Component {"uvw"[k]} spectrum, f = {f} Hz')